In [3]:
import json
from os.path import basename, join

from smart_open import open as smart_open
from dask.distributed import Client
import xarray as xr
import fsspec
from rechunker import rechunk

%matplotlib inline

def get_json(uri):
    with smart_open(uri) as fd:
        return json.load(fd)

client = Client(n_workers=8)        

/opt/conda/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37961 instead
  warnings.warn(


In [4]:
nwm_uri = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'
ds = xr.open_zarr(fsspec.get_mapper(nwm_uri, anon=True, requester_pays=True))

In [8]:
import random
nb_feature_ids = ds.feature_id.shape[0]
rand_inds = list(range(0, nb_feature_ids))
random.shuffle(rand_inds)
test_comids = ds.feature_id.values[rand_inds[0:10]]
time_range = slice('1979-02-01', '1979-02-02')
sub_ds = ds.sel(feature_id=test_comids, time=time_range)
sub_ds

<xarray.Dataset>
Dimensions:     (feature_id: 10, time: 47)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(10,), meta=np.ndarray>
  * feature_id  (feature_id) int32 2899217 19469433 ... 18880454 12604295
    gage_id     (feature_id) |S15 dask.array<chunksize=(10,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(10,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(10,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(10,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 1979-02-02T23:0...
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(47, 1), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(47, 1), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [9]:
# save zarr to disk locally for testing purposes
import os 
import shutil
from os import makedirs
root_dir = '/opt/data/noaa/rechunker-test'
if os.path.isdir(root_dir):
    shutil.rmtree(root_dir)
makedirs(root_dir)

# Fails without this block. See https://github.com/pydata/xarray/issues/5219 
for var in sub_ds:
    del sub_ds[var].encoding['chunks']

target_uri = join(root_dir, 'chrtout-test-06-29-2021.zarr')
sub_ds.to_zarr(target_uri)